## Demo Mask RCNN

In [1]:
!pip install tensorflow-gpu==2.0.0
# %tensorflow_version 2.0.0

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 450kB 45.5MB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=d9c4f649ac0674b6048bcffb0f21192c3df48acc82f37e474bebed96a8282c2b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installa

In [2]:
!pip install keras==2.3.1

     |████████████████████████████████| 378kB 15.7MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Root directory dari project
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/Deep Learning/Tugas Kelompok DL/Tugas Implementasi Mask RCNN/Colab/Mask_RCNN-master")
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # supaya dapat mengimport library mrcnn yang terdapat di folder mrcnn

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

In [ ]:
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

## Setting Konfigurasi
Class Mask RCNN memerlukan objek konfigurasi sebagai parameternya. Objek konfigurasi ini mendefinisikan cara model melakukan training maupun inference

In [ ]:
# define the test configuration
class TestConfig(coco.CocoConfig):
    NAME = "test"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = TestConfig()
config.display()

## Define Instance Model Mask RCNN

In [ ]:
# define modelnya
model_maskrcnn = modellib.MaskRCNN(mode='inference', model_dir='./', config=config)

# Load weights yang sudah di train di dataset MS-COCO
model_maskrcnn.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names
Model Mask RCNN akan mengklasifikasikan objek dan mereturnkan sebuah id integer. Contohnya di MS-COCO Dataset class 'person' adalah 1 dan class 'teddy bear' adalah 88 <br> <br>
Untuk mendapatkan list of class names, dataset di load terlebih dahulu dan menggunakan property ```class_names```, seperti di bawah ini.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Melakukan testing / inference untuk mendeteksi objek

In [ ]:
%matplotlib inline

In [ ]:
# load photograph
image = skimage.io.imread(ROOT_DIR+'/samples/lampu-lalu-lintas.jpg')

# Run detection
results = model_maskrcnn.detect([image], verbose=1)

# Visualize results
r = results[0]
# print(r)
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

## Implementasi Real Time

In [ ]:
def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

In [ ]:
def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

In [ ]:
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

In [ ]:
import cv2

In [ ]:
capture = cv2.VideoCapture(0)

# these 2 lines can be removed if you dont have a 1080p camera.
# capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

while True:
    ret, frame = capture.read()
    results = model_maskrcnn.detect([frame], verbose=0)
    r = results[0]
    frame = display_instances(
        frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
    )
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()